### 08 그룹으로 묶어 연산하기

#### [08 - 1] 데이터 집계하기

##### 1) groupby() 메서드로 데이터 집계하기

In [1]:
import pandas as pd

# 데이터셋 불러오기
df = pd.read_csv('../data/gapminder.tsv', sep = '\t')

In [2]:
# year열을 기준으로 lifeExp 평균 구하기
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [3]:
# year 열의 고윳값
years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [4]:
# 1952년 데이터 확인
y1952 = df.loc[df.year == 1952, :]
print(y1952)

                 country continent  year  lifeExp       pop    gdpPercap
0            Afghanistan      Asia  1952   28.801   8425333   779.445314
12               Albania    Europe  1952   55.230   1282697  1601.056136
24               Algeria    Africa  1952   43.077   9279525  2449.008185
36                Angola    Africa  1952   30.015   4232095  3520.610273
48             Argentina  Americas  1952   62.485  17876956  5911.315053
...                  ...       ...   ...      ...       ...          ...
1644             Vietnam      Asia  1952   40.412  26246839   605.066492
1656  West Bank and Gaza      Asia  1952   43.160   1030585  1515.592329
1668         Yemen, Rep.      Asia  1952   32.548   4963829   781.717576
1680              Zambia    Africa  1952   42.038   2672000  1147.388831
1692            Zimbabwe    Africa  1952   48.451   3080907   406.884115

[142 rows x 6 columns]


In [5]:
# 1952년도의 lifeExp 열의 평균
y1952_mean = y1952["lifeExp"].mean()
print(y1952_mean)

49.057619718309866


##### 1. groupby() 메서드와 함께 사용하는 집계 메서드

In [6]:
# 요약 통계 확인
continent_describe = df.groupby('continent')["lifeExp"].describe()
print(continent_describe)

           count       mean        std     min       25%      50%       75%  \
continent                                                                     
Africa     624.0  48.865330   9.150210  23.599  42.37250  47.7920  54.41150   
Americas   300.0  64.658737   9.345088  37.579  58.41000  67.0480  71.69950   
Asia       396.0  60.064903  11.864532  28.801  51.42625  61.7915  69.50525   
Europe     360.0  71.903686   5.433178  43.585  69.57000  72.2410  75.45050   
Oceania     24.0  74.326208   3.795611  69.120  71.20500  73.6650  77.55250   

              max  
continent          
Africa     76.442  
Americas   80.653  
Asia       82.603  
Europe     81.757  
Oceania    81.235  


##### 2. agg() 메서드와 groupby() 메서드 조합하기

##### 1) 다른 라이브러리의 집계 함수 사용하기

In [7]:
import numpy as np

# agg() 메서드에 넘파이 라이브러리에서 제공하는 함수 전달하여 평균 구하기
cont_le_agg = df.groupby('continent')["lifeExp"].agg(np.mean)
print(cont_le_agg)

continent
Africa      48.865330
Americas    64.658737
Asia        60.064903
Europe      71.903686
Oceania     74.326208
Name: lifeExp, dtype: float64


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\2791477282.py:4: FutureWarning: The provided callable <function mean at 0x000001CA49F53100> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  cont_le_agg = df.groupby('continent')["lifeExp"].agg(np.mean)


##### 2) 사용자 집계 함수 사용하기

In [8]:
# 평균 구하는 함수 만들기
def my_mean(values):
    n = len(values)         ## 숫자 개수 구하기
    sum = 0                 ## 합계를 0으로 초기화
    for value in values:
        sum += value        ## 덧셈
    return sum / n      ## 합계를 숫자 개수로 나눈 값 반환

In [9]:
# 평균 구하기
agg_my_mean = df.groupby('year')['lifeExp'].agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [10]:
# 매개변수가 여러 개인 사용자 함수 agg()에 전달
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return (mean - diff_value)

In [11]:
# 모든 기대 수명의 평균
global_mean = df['lifeExp'].mean()
print(global_mean)

59.474439366197174


In [12]:
# 각 연도별 기대수명 평균과 전체 기대 수명의 차이
agg_mean_diff = (
    df.
    groupby("year")
    ["lifeExp"]
    .agg(my_mean_diff, diff_value = global_mean)
)
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


##### 3) 여러 개의 집계 함수 한 번에 사용하기

In [13]:
# 여러 개의 집계 함수 한 번에 사용
gdf = (
    df
    .groupby('year')
    ['lifeExp']
    .agg([np.count_nonzero, np.mean, np.std])
)
print(gdf)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\2901945109.py:6: FutureWarning: The provided callable <function mean at 0x000001CA49F53100> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg([np.count_nonzero, np.mean, np.std])
C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\2901945109.py:6: FutureWarning: The provided callable <function std at 0x000001CA49F53240> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg([np.count_nonzero, np.mean, np.std])


##### 5) agg()나 aggregate() 메서드에 딕셔너리 사용하기

In [14]:
# 딕셔너리에 함수 정보 담아 agg()나 aggregate() 메서드에 전달
## 데이터프레임에 사용
gdf_dict = df.groupby('year').agg(
    {"lifeExp" : "mean",        ## 데이터프레임이라면 {"열 이름" : "함수"} 형식의 딕셔너리를 전달
     "pop" : "median",
     "gdpPercap" : "median"}
)
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


In [16]:
## 시리즈에 사용
gdf = (
    df.groupby("year")['lifeExp']
    .agg(           ## 시리즈에서는 먼저 집계
        [np.count_nonzero, np.mean, np.std]
    )
    .rename(        ## 집계하고 나서 열 이름을 바꿈
        columns = {
            "count_nonzero" : "count",
            "mean" : "avg",
            "std" : "std_dev",
        }
    )
. reset_index()        ## 평탄화한 데이터프레임 반환
)
print(gdf)

    year  count        avg    std_dev
0   1952    142  49.057620  12.225956
1   1957    142  51.507401  12.231286
2   1962    142  53.609249  12.097245
3   1967    142  55.678290  11.718858
4   1972    142  57.647386  11.381953
5   1977    142  59.570157  11.227229
6   1982    142  61.533197  10.770618
7   1987    142  63.212613  10.556285
8   1992    142  64.160338  11.227380
9   1997    142  65.014676  11.559439
10  2002    142  65.694923  12.279823
11  2007    142  67.007423  12.073021


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\2879009367.py:4: FutureWarning: The provided callable <function mean at 0x000001CA49F53100> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(           ## 시리즈에서는 먼저 집계
C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\2879009367.py:4: FutureWarning: The provided callable <function std at 0x000001CA49F53240> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(           ## 시리즈에서는 먼저 집계


#### [08 - 2] 데이터 변환하기

##### 1. 표준점수 계산하기

In [17]:
# 표준점수 계산 함수 만들기
def my_zscore(x):   ## 매개변수 x : 값의 시리즈 또는 벡터
    return ((x - x.mean()) / x.std())   ## 판다스의 평균 함수와 표준편차 함수로 표준점수 구함

In [18]:
# transform() 메서드 사용하여 my_zscore() 함수로 year 열의 lifeExp 변환
transform_z = df.groupby('year')['lifeExp'].transform(my_zscore)
print(transform_z)

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64


In [19]:
# 원본과 변환한 데이터프레임 비교
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


In [20]:
from scipy.stats import zscore

In [24]:
# 그룹화후 transfrom() 메서드로 zoscore() 적용
sp_z_grouped = df.groupby('year')['lifeExp'].transform(zscore)

# 그룹화하지 않고 zscore() 적용
sp_z_nogroup = zscore(df['lifeExp'])

# 결과 확인 비교
print(transform_z.head())
print('----------------')
print(sp_z_grouped.head())
print('----------------')
print(sp_z_nogroup[:5])

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64
----------------
0   -1.662719
1   -1.737377
2   -1.792867
3   -1.854699
4   -1.900878
Name: lifeExp, dtype: float64
----------------
[-2.37533395 -2.25677417 -2.1278375  -1.97117751 -1.81103275]


##### 2. 평균값으로 결측값 채우기

In [25]:
import seaborn as sns
import numpy as np

np.random.seed(42)      ## 실행할 때마다 같은 결과를 얻도록 설정

# seaborn 라이브러리의 tips 데이터셋 10개 행 추출
tips_10 = sns.load_dataset("tips").sample(10)

In [27]:
# 추출한 10개 행 중에 4개의 total_bill 값을 무작위로 선택하여 결측값 np.NaN으로 변경
tips_10.loc[np.random.permutation(tips_10.index)[:4], "total_bill"] = np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24          NaN  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124         NaN  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [30]:
# sex 열의 각 값에서 결측값이 아닌 값의 개수 확인
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             3    7       7    7     7     7
Female           1    3       3    3     3     3


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\4114725263.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_sex = tips_10.groupby('sex').count()


In [ ]:
# 그룹화된 평균 계산 후 결측값 채우기 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

total_bill_group_mean = (
    tips_10.groupby('sex').total_bill.transform(fill_na_mean)
)

C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\3310670630.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips_10.groupby('sex').total_bill.transform(fill_na_mean)


In [33]:
# 원본 데이터에 결측값 채운 새로운 열 추가
tips_10["fill_total_bill"] = total_bill_group_mean

In [34]:
# 결과 확인
print(tips_10[['sex', 'total_bill', 'fill_total_bill']])

        sex  total_bill  fill_total_bill
24     Male         NaN            17.33
6      Male        8.77             8.77
153    Male         NaN            17.33
211    Male         NaN            17.33
198  Female         NaN            15.38
176    Male         NaN            17.33
192    Male       28.44            28.44
124  Female         NaN            15.38
9      Male       14.78            14.78
101  Female       15.38            15.38


#### [08 - 3] 원하는 데이터 걸러 내기

##### 1) 데이터 필터링하기

In [36]:
tips = sns.load_dataset('tips')
print(tips.shape)
print(tips.head())

(244, 7)
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [38]:
# size 열 각 값의 빈도수
print(tips['size'].value_counts())

size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64


In [41]:
# 30개 이상인 데이터만 필터링
tips_flitered = (tips.groupby('size').filter(lambda x : x['size'].count() >= 30))

# 결과 확인
print(tips_flitered.shape)
print(tips_flitered['size'].value_counts())

(231, 7)
size
2    156
3     38
4     37
Name: count, dtype: int64


#### [08 - 4] 그룹 객체 활용하기

In [43]:
# tips 데이터셋의 sample() 함수로 10개 행 무작위로 추출
tips_10 = sns.load_dataset('tips').sample(10, random_state = 42)    ## random_state를 고정값으로 지정하면 실행할 때마다 동일한 결과 얻음
print(tips_10) 

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [45]:
# goupby() 메서드 호출 결과 확인
grouped = tips_10.groupby('sex')
print(grouped)

C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\823447533.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips_10.groupby('sex')


In [47]:
# 그룹화한 데이터 확인
print(grouped.groups)   ## 행 번호 추출

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


##### 1) 그룹 객체로 여러 열에 집계 함수 적용하기

In [49]:
# 숫자인 열만 평균 구하기
avgs = grouped.mean(numeric_only = True)
print(avgs)

## 특정 열을 지정하지 않고 groupby() 뒤에 바로 계산을 수행하면 해당 계산을 적용할 수 있는 열만 계산하고 나머지 열은 건너뜀

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


##### 2) 그룹 추출하고 순회하기

In [ ]:
# 원하는 그룹 추출
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [51]:
# for문으로 순회
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [53]:
# grouped 객체의 첫 번째 인덱스 추출
# print(grouped[0])     ## 오류 : 판다스의 DataFrameGroupBy 객체이기 때문

In [67]:
# for문에 break문 추가하여 첫 번째 그룹만 출력
for sex_group in grouped:

    ## 객체 자료형 : 튜플
    print(f'the type is : {type(sex_group)}\n')    

    ## 객체 길이 : 2
    print(f'the length is : {len(sex_group)}\n')     
    
    ## 첫 번째 요소
    first_element = sex_group[0]
    print(f'the first element is : {first_element}\n')

    ## 첫 번째 요소의 자료형 : 문자열
    print(f'it has a type of : {type(sex_group[0])}\n')

    ## 두 번째 요소
    second_element = sex_group[1]
    print(f'the second element is : \n{second_element}\n')

    ## 두 번째 요소의 자료형 : 데이터프레임
    print(f'it has a type of : {type(second_element)}\n')

    ## 그룹 출력
    print(f'what we have : ')
    print(sex_group)

    ## for문 중단
    break;


the type is : <class 'tuple'>

the length is : 2

the first element is : Male

it has a type of : <class 'str'>

the second element is : 
     total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of : <class 'pandas.core.frame.DataFrame'>

what we have : 
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192     

##### 3) 여러 개의 변수로 그룹화하고 결과 평탄화하기

In [70]:
# 성별(sex), 시간(time)별로 tips 데이터의 평균 구하기
bill_sex_time = tips_10.groupby(['sex', 'time'])

group_avg = bill_sex_time.mean(numeric_only = True)

# 결과 확인
print(group_avg)
print(type(group_avg))

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000
<class 'pandas.core.frame.DataFrame'>


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\1882435489.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bill_sex_time = tips_10.groupby(['sex', 'time'])


In [ ]:
# 열 이름 출력
print(group_avg.columns)

# 행 인덱스 출력
print(group_avg.index)      ## MultiIndex : 여러 인덱스를 담은 인덱스

Index(['total_bill', 'tip', 'size'], dtype='object')
MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [74]:
# 평탄화 방법 1
group_method = tips_10.groupby(['sex', 'time']).mean(numeric_only = True).reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\1822891737.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_method = tips_10.groupby(['sex', 'time']).mean(numeric_only = True).reset_index()


In [76]:
# 평탄화 방법 2
group_param = tips_10.groupby(['sex', 'time'], as_index = False).mean(numeric_only = True)
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


C:\Users\clatk\AppData\Local\Temp\ipykernel_4728\3847756176.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_param = tips_10.groupby(['sex', 'time'], as_index = False).mean(numeric_only = True)
